In [12]:
from megatron.tokenizer.tokenizer import build_debug
import json
vocab_f='/workspace/SVdata/bpe/32k/vocab.json'
merge_f='/workspace/SVdata/bpe/32k/merges.txt'

tokenizer=build_debug(vocab_f,merge_f,'bpe')
tokenizer.tokenize('[UNK]')
with open(vocab_f, 'r') as f2:
    vocab = json.loads(f2.read())
rev_vocab={v: k for k, v in vocab.items()}
rev_vocab[6]

'[UNK]'

In [ ]:
tokenizer.decode_token_ids(6)

In [ ]:
len(labels)

In [1]:
import os , sys 
import numpy as np
import json
f_dev=open('dev_predict.csv','r')
f_test=open('test_predict.csv','r')
dev=json.loads(f_dev.read())
test=json.loads(f_test.read())


In [2]:
ids=test['id']
probs=test['softmax']
labels=test['label']
len(labels) , ids[:5] ,min(ids), max(ids)
labels=[l for l in labels]
labels[-10:], ids[-10:], min(ids), max(ids), len(set(ids)), set(labels), type(labels[0])

([2, 1, 0, 2, 3, 2, 3, 1, 3, 3],
 [42506, 42704, 41777, 39410, 40772, 39067, 39063, 39206, 41248, 39019],
 38640,
 43469,
 4830,
 {0, 1, 2, 3, 4},
 int)

In [3]:
import pandas as pd
dev_df=pd.read_csv('/workspace/SVdata/SuperLimSE/sentiment_processed/SV_sentiment_dev.tsv',sep='\t')
test_df=pd.read_csv('/workspace/SVdata/SuperLimSE/sentiment_processed/SV_sentiment_test.tsv',sep='\t')
test_df.iloc[0,-1], round(float(test_df.iloc[0,-1])), test_df.columns

(3.0, 3, Index(['index', 'doc', 'par', 'text', 'label'], dtype='object'))

In [4]:
indexes=test_df.index
len(set(indexes)), min(indexes), max(indexes) , type(ids), type(labels[0]), type(probs)

(4830, 0, 4829, list, int, list)

In [5]:
def return_index(idx,df):
    indexes=list(df['index'].values)
    return indexes.index(idx)


In [6]:
test_df['predicted_proba']=None
test_df['predicted_label']=None
test_df['correct_flag']=None

In [8]:
LABELS = {'1': 0, '2': 1 , '3' :2 ,'4':3 ,'5':4}
num2LABELS = dict([(v,int(k)) for (k,v) in LABELS.items()])
incorrect=0
n=len(test_df)
for i in range(n):
    idx=ids[i]
    #print(idx,i)
    #pred=labels[i]
    pred_proba=probs[i]
    pred=np.argmax(pred_proba)
    map2index=return_index(idx,test_df)
    actual=str(round(float(test_df.iloc[map2index,-4])))    
    actual=LABELS[actual]
    if pred==actual:        
        test_df.iloc[map2index,-2]=pred+1
        test_df.at[map2index,'predicted_proba']=pred_proba
        test_df.iloc[map2index,-1]='correct'
        #print("predicted_label={} , actual={}".format(predicted_label, actual))        
    else:
        print("processing id =",idx, i)
        print("predicted_label={} , actual={}".format(pred, actual))
       
        test_df.iloc[map2index,-2]=pred+1
        test_df.iloc[map2index,-1]='incorrect'
        test_df.at[map2index,'predicted_proba']=pred_proba
        incorrect+=1
        #print("-----"*10)    
print(incorrect, incorrect/len(test_df))

processing id = 43160 335
predicted_label=1 , actual=0
processing id = 42037 417
predicted_label=2 , actual=1
processing id = 42313 610
predicted_label=1 , actual=0
processing id = 39109 658
predicted_label=3 , actual=1
processing id = 43156 664
predicted_label=2 , actual=0
processing id = 40650 675
predicted_label=2 , actual=3
processing id = 42537 695
predicted_label=3 , actual=4
processing id = 40629 778
predicted_label=1 , actual=0
processing id = 43119 847
predicted_label=3 , actual=4
processing id = 40408 868
predicted_label=1 , actual=0
processing id = 42862 917
predicted_label=2 , actual=1
processing id = 41126 935
predicted_label=1 , actual=3
processing id = 42553 1167
predicted_label=1 , actual=0
processing id = 38869 1223
predicted_label=2 , actual=1
processing id = 41937 1319
predicted_label=3 , actual=4
processing id = 40733 1367
predicted_label=3 , actual=4
processing id = 38954 1414
predicted_label=1 , actual=0
processing id = 41378 1440
predicted_label=2 , actual=1
proc

In [9]:

test_df.head()

,index,doc,par,text,label,predicted_proba,predicted_label,correct_flag
0,38640,z02679_flashback-50294563,3,Förklarar regeringen och tidigare regeringars ...,3.0,"[0.00024160165048670024, 0.004885985516011715,...",3,correct
1,38641,z02679_flashback-50294563,5,Kom gärna med förslag på olika samband mellan ...,3.0,"[0.0005016974173486233, 0.002114288741722703, ...",3,correct
2,38642,z01476_flashback-56524490,1,Sossarnas invandringspolitik bidrar till ännu ...,2.0,"[0.004491564352065325, 0.9182902574539185, 0.0...",2,correct
3,38643,z01476_flashback-56524490,3,Sossarna står ju främst för att dom vill ha et...,2.0,"[0.017512038350105286, 0.9626832604408264, 0.0...",2,correct
4,38644,z01476_flashback-56524490,10,Den nya invandrarklassen har fått det mycket s...,2.0,"[0.010351656004786491, 0.980188250541687, 0.00...",2,correct


In [10]:
test_df.to_csv('/workspace/SVdata/SuperLimSE/sentiment_processed/pred/SV_sentiment_test_Addpredicted.tsv', index=False, sep='\t' )

In [17]:
test_df[dev_df['correct_flag']=='incorrect']

,index,doc,par,text,label,predicted_proba,predicted_label,correct_flag
22,38662,z02473_flashback-64194218,1,Alternativ för Sverige har blockerats från att...,3.0,"[0.0005994983366690576, 0.0024945770855993032,...",3,correct
23,38663,z02473_flashback-64194218,2,Som rubriken lyder har AfS blockerats från att...,3.0,"[0.00020217729615978897, 0.0005988927441649139...",3,correct
39,38679,z01932_flashback-51348725,17,"Dessa fyra somalier började då att knuffa mig,...",2.0,"[0.020440692082047462, 0.7163495421409607, 0.0...",2,correct
48,38688,z02028_flashback-58919294,4,När stackars Mohamad hämtat sig från chocken u...,2.0,"[0.005265498999506235, 0.9787473678588867, 0.0...",2,correct
329,38969,z01960_flashback-57909360,4,Upplever ni också dom här orättvisorna?.,1.0,"[0.9801287055015564, 0.01625763811171055, 0.00...",1,correct
348,38988,z00584_flashback-21555951,1,"Senaste TV-snyftaren: Somaliska flykting-""barn...",2.0,"[0.12021160125732422, 0.8710700273513794, 0.00...",2,correct
534,39174,z01311_flashback-2366290,18,Värt att tänka på när man ska välja i valet. K...,2.0,"[0.022020351141691208, 0.9499139189720154, 0.0...",2,correct
557,39197,z03257_flashback-50319585,6,De präglas oerhört hårt av religion och det är...,1.0,"[0.8211489319801331, 0.11754918843507767, 0.02...",1,correct
605,39245,z04053_flashback-55752359,6,Vad tror ni kommer hända härnäst med dessa sta...,2.0,"[0.039388060569763184, 0.9580203294754028, 0.0...",2,correct
606,39246,z00003_flashback-3638566,1,Integration kanske omöjlig.,2.0,"[0.021400416269898415, 0.9524466395378113, 0.0...",2,correct


In [11]:
ids=dev['id']
probs=dev['softmax']
labels=dev['label']
len(labels) , ids[:5] ,min(ids), max(ids)
labels=[l for l in labels]
labels[-10:], ids[-10:], min(ids), max(ids), len(set(ids)), set(labels), type(labels[0])

([3, 2, 1, 3, 2, 2, 2, 2, 1, 2],
 [47336, 47534, 46607, 44240, 45602, 43897, 43893, 44036, 46078, 43849],
 43470,
 48299,
 4830,
 {0, 1, 2, 3, 4},
 int)

In [12]:
dev_df['predicted_proba']=None
dev_df['predicted_label']=None
dev_df['correct_flag']=None

In [13]:
LABELS = {'1': 0, '2': 1 , '3' :2 ,'4':3 ,'5':4}
num2LABELS = dict([(v,int(k)) for (k,v) in LABELS.items()])
incorrect=0
n=len(dev_df)
for i in range(n):
    idx=ids[i]
    #print(idx,i)
    #pred=labels[i]
    pred_proba=probs[i]
    pred=np.argmax(pred_proba)
    map2index=return_index(idx,dev_df)
    actual=str(round(float(dev_df.iloc[map2index,-4])))    
    actual=LABELS[actual]
    if pred==actual:        
        dev_df.iloc[map2index,-2]=pred+1
        dev_df.at[map2index,'predicted_proba']=pred_proba
        dev_df.iloc[map2index,-1]='correct'
        #print("predicted_label={} , actual={}".format(predicted_label, actual))        
    else:
        print("processing id =",idx, i)
        print("predicted_label={} , actual={}".format(pred, actual))
       
        dev_df.iloc[map2index,-2]=pred+1
        dev_df.iloc[map2index,-1]='incorrect'
        dev_df.at[map2index,'predicted_proba']=pred_proba
        incorrect+=1
        #print("-----"*10)    
print(incorrect, incorrect/len(dev_df))

processing id = 43799 62
predicted_label=1 , actual=3
processing id = 44075 162
predicted_label=3 , actual=4
processing id = 47481 249
predicted_label=3 , actual=4
processing id = 45916 264
predicted_label=1 , actual=3
processing id = 47398 267
predicted_label=2 , actual=3
processing id = 44112 291
predicted_label=2 , actual=0
processing id = 44116 600
predicted_label=1 , actual=0
processing id = 47323 716
predicted_label=2 , actual=3
processing id = 47410 776
predicted_label=1 , actual=0
processing id = 44004 898
predicted_label=3 , actual=4
processing id = 47850 1010
predicted_label=1 , actual=0
processing id = 46645 1113
predicted_label=1 , actual=0
processing id = 47383 1167
predicted_label=0 , actual=1
processing id = 46267 1303
predicted_label=3 , actual=2
processing id = 44844 1333
predicted_label=3 , actual=4
processing id = 48007 1420
predicted_label=2 , actual=4
processing id = 44437 1429
predicted_label=2 , actual=0
processing id = 46102 1453
predicted_label=2 , actual=1
pro

In [14]:
dev_df.to_csv('/workspace/SVdata/SuperLimSE/sentiment_processed/pred/SV_sentiment_dev_Addpredicted.tsv', index=False, sep='\t' )

In [15]:
dev_df.head()

,index,doc,par,text,label,predicted_proba,predicted_label,correct_flag
0,43470,www_aftonbladet_se_debatt_a_gPG3xq_sd-har-fatt...,26,Respekt för mänskliga rättigheter är ett gemen...,4.333333,"[0.00030437082750722766, 0.0004385532229207456...",4,correct
1,43471,www_aftonbladet_se_debatt_a_gPG3xq_sd-har-fatt...,4,Mot enskilda mänskors strävan att ge sig själv...,3.428571,"[0.00011491846817079931, 0.0007807372021488845...",3,correct
2,43472,www_aftonbladet_se_debatt_a_gPG3xq_sd-har-fatt...,8,► ”Integrationspolitiken är ett totalt misslyc...,2.750000,"[0.00012359624088276178, 0.0002970716159325093...",3,correct
3,43473,www_aftonbladet_se_debatt_a_gPG3xq_sd-har-fatt...,15,Sverige har haft reglerad invandring under lån...,3.200000,"[0.00012999471800867468, 0.0012849941849708557...",3,correct
4,43474,www_aftonbladet_se_debatt_a_gPG3xq_sd-har-fatt...,17,Det avskyvärda kriget i Syrien har genererat s...,3.800000,"[0.0003444357425905764, 0.0006836557295173407,...",4,correct


In [16]:
dev_df[dev_df['correct_flag']=='incorrect']

,index,doc,par,text,label,predicted_proba,predicted_label,correct_flag
22,43492,z02748_flashback-53813668,1,Bosätta sig i Mexico?,5.000000,"[0.2551259696483612, 0.10730428248643875, 0.48...",3,incorrect
23,43493,z02748_flashback-53813668,2,Någon som har tips på hur man ska gå tillväga ...,5.000000,"[0.00219488306902349, 0.0143684521317482, 0.00...",4,incorrect
39,43509,z01161_flashback-58942871,6,Hur många miljarder ska man slösa bort denna g...,1.000000,"[0.4232015609741211, 0.5381207466125488, 0.032...",2,incorrect
48,43518,z00266_flashback-18808599,7,Därtill är det välkänt för alla och envar att ...,1.000000,"[0.41447651386260986, 0.5765686631202698, 0.00...",2,incorrect
329,43799,z02145_flashback-60355080,3,Fick höra idag att en kille jag känner har fåt...,4.000000,"[0.004899358376860619, 0.8207331299781799, 0.0...",2,incorrect
348,43818,z02631_flashback-58197849,9,"1. Är inte detta med den ""svenska modellen"" en...",2.000000,"[0.04270680248737335, 0.1354571133852005, 0.81...",3,incorrect
534,44004,www_aftonbladet_se_debatt_debattamnen_varlden_...,7,För det tredje måste vi stå upp för en human f...,4.600000,"[0.004180471878498793, 0.0034997437614947557, ...",4,incorrect
557,44027,z00808_flashback-55141651,4,"Ju fler ""flyktingar"" som kommer, desto mer ver...",1.000000,"[0.26903364062309265, 0.7019209265708923, 0.02...",2,incorrect
605,44075,z02223_www_svd_se_stang-inte-granserna-for-syr...,5,Konflikten i Syrien har resulterat i en av de ...,5.000000,"[0.003213798627257347, 0.006485715042799711, 0...",4,incorrect
606,44076,z02223_www_svd_se_stang-inte-granserna-for-syr...,6,UNHCR anser att flyktingar från Syrien är i be...,5.000000,"[0.002652756404131651, 0.02796819619834423, 0....",4,incorrect
